In [ ]:
#(For phone calls) ADB must be installed and usb debugging must be ON on the device
%pip install google-generativeai
import os
import json
import platform
import difflib
import webbrowser
import urllib.parse
import random
import tkinter as tk
from tkinter import messagebox, filedialog, ttk, Toplevel, Listbox, Scrollbar, END
import smtplib
from email.message import EmailMessage
from datetime import datetime
import google.generativeai as genai


In [ ]:
def match_command(text, keywords):
    text = text.lower()
    for keyword in keywords:
        return any(keyword in text for keyword in keywords)

**Phone Call**

In [ ]:
contacts = []
def PhoneCall():
    def get_contact_number(identifier):
        if identifier.isdigit():
            return identifier
        else:
            for name, number in contacts:
                if name.lower() == identifier.lower():
                    return number
            return None

    def make_call():
        identifier = entry_contact.get().strip()
        if not identifier:
            messagebox.showerror("Input Error", "Please enter a name or number.")
            return

        number = get_contact_number(identifier)
        if not number:
            messagebox.showerror("Not Found", f"No contact found for '{identifier}'")
            return

        command = f'adb shell am start -a android.intent.action.CALL -d tel:{number}'
        result = os.system(command)

        if result == 0:
            messagebox.showinfo("Success", f"Calling {number}...")
        else:
            messagebox.showerror("Error", "Failed to initiate call. Check USB debugging and ADB.")

    def save_contact():
        name = entry_name.get().strip()
        number = entry_contact.get().strip()

        if not name or not number.isdigit():
            messagebox.showerror("Invalid Input", "Enter a valid name and phone number.")
            return

        # Check for duplicates
        for existing_name, _ in contacts:
            if existing_name.lower() == name.lower():
                messagebox.showerror("Duplicate", f"Contact '{name}' already exists.")
                return

        contacts.append((name, number))
        messagebox.showinfo("Saved", f"Contact '{name}' saved successfully.")

    def show_contacts():
        if not contacts:
            messagebox.showinfo("No Contacts", "No contacts added yet.")
            return

        contact_window = Toplevel(root)
        contact_window.title("Saved Contacts")
        
        scrollbar = Scrollbar(contact_window)
        scrollbar.pack(side="right", fill="y")

        contact_list = Listbox(contact_window, font=("Arial", 10), yscrollcommand=scrollbar.set)
        contact_list.pack(fill="both", expand=True)
        scrollbar.config(command=contact_list.yview)

        for name, number in contacts:
            contact_list.insert(END, f"{name}: {number}")
        
    # GUI setup
    root = tk.Tk()
    root.title("PHONE")


    tk.Label(root, text="Contact Name (for saving):", font=("Arial", 12)).pack(pady=(10, 0))
    entry_name = tk.Entry(root, font=("Arial", 12), justify='center')
    entry_name.pack(pady=5)

    tk.Label(root, text="Phone Number or Name to Call:", font=("Arial", 12)).pack()
    entry_contact = tk.Entry(root, font=("Arial", 14), justify='center')
    entry_contact.pack(pady=5)

    call_button = tk.Button(root, text="Make Call", font=("Arial", 12), command=make_call)
    call_button.pack(pady=5)

    save_button = tk.Button(root, text="Save Contact", font=("Arial", 12), command=save_contact)
    save_button.pack(pady=5)

    show_button = tk.Button(root, text="Show Contacts", font=("Arial", 12), command=show_contacts)
    show_button.pack(pady=10)

    root.mainloop()

**Base Media Player Class**

In [ ]:
class MediaPlayer:
    def __init__(self, folder_path, valid_extensions):
        self.folder_path = folder_path
        self.valid_extensions = valid_extensions
        self.files = self._scan_files()
      

    def _scan_files(self):
        found_files = []
        for root, _, files in os.walk(self.folder_path): 
            for file in files:
                if os.path.splitext(file)[1].lower() in self.valid_extensions:
                    full_path = os.path.join(root, file)
                    found_files.append(os.path.relpath(full_path, self.folder_path))
        return found_files
    

    def list_files(self):
        if not self.files:
            print("No media files found.")
            return
        print("\nAvailable Files:")
        for index, file in enumerate(self.files, 1):
            print(f"   {index}. {file}")


    def suggest_filename(self, user_input):
        matches = difflib.get_close_matches(user_input, self.files, n=1, cutoff=0.4)
        return matches[0] if matches else None


    def open_file(self, filename):
        full_path = os.path.abspath(os.path.join(self.folder_path, filename))
        if not os.path.exists(full_path):
            print("File does not exist.")
            return
        system = platform.system()
        print(f"Opening: {full_path}")
        if system == "Windows":
            os.startfile(full_path)
        elif system == "Darwin":
            os.system(f"open '{full_path}'")
        else:
            os.system(f"xdg-open '{full_path}'")

**Music Player**

In [ ]:
class MusicPlayer(MediaPlayer):
    def __init__(self, folder_path):
        super().__init__(folder_path, ['.mp3', '.wav', '.aac', '.flac', '.ogg', '.m4a'])

    def play_music(self):
        self.list_files()
        if not self.files:
            return
        
        user_input = input("\nEnter song name (partial/full) or type 'shuffle' to shuffle songs: ").strip()

        if user_input == "shuffle":
            random_song = random.choice(self.files)
            print(f"Shuffled: {random_song}")
            self.open_file(random_song)
            return
    
        match = self.suggest_filename(user_input)
        if match:
            print(f"Best match: {match}")
            self.open_file(match)
        else:
            print("No close match found.")

**Video Player**

In [ ]:
class VideoPlayer(MediaPlayer):
    def __init__(self, folder_path):
        super().__init__(folder_path, ['.mp4', '.mkv', '.avi', '.mov', '.flv', '.webm'])

    def play_video(self):
        self.list_files()
        if not self.files:
            return
        user_input = input("\nEnter video name (partial/full): ").strip()
        match = self.suggest_filename(user_input)
        if match:
            print(f"Best match: {match}")
            self.open_file(match)
        else:
            print("No close match found.")

**Youtube Search**

In [ ]:
# YouTube Search
class YouTube:
    def search(self):
        query = input("\n What do you want to search on YouTube? ").strip()
        if not query:
            print("Empty search is not allowed.")
            return
        search_url = "https://www.youtube.com/results?search_query=" + urllib.parse.quote(query)
        print(f"Opening YouTube search for: {query}")
        webbrowser.open(search_url)


**Link Opener**

In [ ]:
HISTORY_FILE = "url_history.json"

# Load last URL from history
def load_last_url():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, 'r') as f:
            data = json.load(f)
            return data.get("last_url", "")
    return ""

# Save URL to history
def save_url(url):
    with open(HISTORY_FILE, 'w') as f:
        json.dump({"last_url": url}, f)

# Validate and fix URL
def validate_url(url):
    url = url.strip()
    if not url.startswith("http://") and not url.startswith("https://"):
        url = "https://" + url
    return url

# Open the URL
def open_url():
    url = validate_url(url_entry.get())
    mode = mode_var.get()

    if not url or url.isspace():
        messagebox.showerror("Error", "Please enter a valid URL.")
        return

    save_url(url)

    try:
        if mode == "tab":
            webbrowser.open_new_tab(url)
        elif mode == "window":
            webbrowser.open_new(url)
        else:
            messagebox.showwarning("Warning", "Please select how to open the URL.")
            return
        messagebox.showinfo("Opened", f"Successfully opened:\n{url}")
    except Exception as e:
        messagebox.showerror("Failed", f"Could not open URL:\n{e}")

# Toggle dark mode
def toggle_dark_mode():
    global dark_mode
    dark_mode = not dark_mode
    bg = "#222" if dark_mode else "#f0f0f0"
    fg = "#fff" if dark_mode else "#000"

    root.configure(bg=bg)
    for widget in root.winfo_children():
        widget.configure(bg=bg, fg=fg)
    style.configure('TButton', background=bg, foreground=fg)
    style.configure('TRadiobutton', background=bg, foreground=fg)
    dark_button.configure(text="☀ Light Mode" if dark_mode else "🌙 Dark Mode")

# Main GUI
root = tk.Tk()
root.title("🌐 Open Website")
root.geometry("420x280")
style = ttk.Style()
dark_mode = False

# Widgets
url_label = tk.Label(root, text="Enter Website URL:", font=('Arial', 12))
url_label.pack(pady=5)

url_entry = tk.Entry(root, width=50)
url_entry.pack(pady=5)
url_entry.insert(0, load_last_url())  # Load previous URL

mode_var = tk.StringVar()
mode_label = tk.Label(root, text="Open in:", font=('Arial', 11))
mode_label.pack(pady=5)

tab_radio = tk.Radiobutton(root, text="New Tab", variable=mode_var, value="tab")
tab_radio.pack()
window_radio = tk.Radiobutton(root, text="New Window", variable=mode_var, value="window")
window_radio.pack()

open_button = tk.Button(root, text="Open URL", command=open_url, bg="green", fg="white", font=('Arial', 12))
open_button.pack(pady=10)

dark_button = tk.Button(root, text="🌙 Dark Mode", command=toggle_dark_mode, font=('Arial', 10))
dark_button.pack()

root.mainloop()

**Email**

In [ ]:
HISTORY_FILE = "email_history.json"
LAST_EMAIL_FILE = "last_email.txt"
DARK_MODE = True

BG_COLOR = "#669BBC" if DARK_MODE else "#f0f4f8"
FG_COLOR = "#ecf0f1" if DARK_MODE else "#2c3e50"
ENTRY_BG = "#34495e" if DARK_MODE else "white"
BTN_COLOR = "#780000"

# Gemini Setup
def setup_gemini(api_key):
    genai.configure(api_key=api_key)

def generate_gemini_message(name, topic, tone):
    prompt = f"Write a {tone.lower()} email to {name} about '{topic}'. Sign it as 'Your Automated Assistant'."
    try:
        model = genai.GenerativeModel(model_name="gemini-pro")
        response = model.generate_content([prompt])  # <-- Important fix
        return response.text.strip()
    except Exception as e:
        return f"Error: {e}"

# Email Sending
def send_email():
    name = name_entry.get()
    to_email = to_email_entry.get()
    topic = topic_entry.get()
    from_email = your_email_entry.get()
    app_password = app_pass_entry.get()
    gemini_key = gemini_key_entry.get()
    tone = tone_var.get()
    subject = f"Regarding {topic.title()}"
    attachment_path = attachment_label['text']

    if not all([name, to_email, topic, from_email, app_password, gemini_key]):
        messagebox.showerror("Error", "All fields except attachment are required.")
        return

    setup_gemini(gemini_key)
    body = generate_gemini_message(name, topic, tone)

    if body.startswith("Error:"):
        messagebox.showerror("Gemini Error", body)
        return

    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    msg.set_content(body)

    if attachment_path and os.path.isfile(attachment_path):
        try:
            with open(attachment_path, 'rb') as f:
                file_data = f.read()
                file_name = os.path.basename(attachment_path)
                msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)
        except Exception as e:
            messagebox.showerror("Attachment Error", f"Failed to attach file: {e}")
            return

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(from_email, app_password)
            smtp.send_message(msg)

        save_to_history(name, to_email, topic, tone, body)
        export_last_email(subject, body)
        messagebox.showinfo("Success", "✅ Email sent successfully!")
    except Exception as e:
        messagebox.showerror("SMTP Error", f"❌ Failed to send email: {e}")

def save_to_history(name, to_email, topic, tone, message):
    record = {
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "to": to_email,
        "name": name,
        "topic": topic,
        "tone": tone,
        "message": message
    }
    history = []
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, "r") as f:
            history = json.load(f)
    history.append(record)
    with open(HISTORY_FILE, "w") as f:
        json.dump(history, f, indent=2)

def export_last_email(subject, body):
    with open(LAST_EMAIL_FILE, "w") as f:
        f.write(f"Subject: {subject}\n\n{body}")

def choose_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        attachment_label.config(text=file_path)

# GUI
root = Tk()
root.title("Email Sender")
root.geometry("650x600")
root.configure(bg=BG_COLOR)

Label(root, text="Email Sender", font=("Helvetica", 20, "bold"), bg=BG_COLOR, fg=FG_COLOR).pack(pady=15)

def field(label_text, is_pass=False):
    frame = Frame(root, bg=BG_COLOR)
    frame.pack(pady=5)
    Label(frame, text=label_text, font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
    entry = Entry(frame, width=50, font=("Helvetica", 11), bg=ENTRY_BG, fg=FG_COLOR, show="*" if is_pass else "")
    entry.pack()
    return entry

name_entry = field("Recipient Name:")
to_email_entry = field("Recipient Email:")
topic_entry = field("Email Topic:")
your_email_entry = field("Your Gmail:")
app_pass_entry = field("Gmail Password:", is_pass=True)
gemini_key_entry = field("API Key:", is_pass=True)

# Tone dropdown
tone_var = StringVar(value="friendly")
tone_frame = Frame(root, bg=BG_COLOR)
tone_frame.pack(pady=5)
Label(tone_frame, text="Select Tone:", font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
ttk.Combobox(tone_frame, textvariable=tone_var, values=["friendly", "formal", "flirty", "funny", "strict"], width=47).pack()

# Attachment chooser
Button(root, text="📎 Choose Attachment", command=choose_file, bg="#3498db", fg="white", font=("Helvetica", 10)).pack(pady=5)
attachment_label = Label(root, text="", bg=BG_COLOR, fg="#bdc3c7", font=("Helvetica", 9))
attachment_label.pack()

# Send button
Button(root, text="🚀 Send Email", command=send_email, bg=BTN_COLOR, fg="white", font=("Helvetica", 13, "bold"), width=30).pack(pady=20)

root.mainloop()

**Smart Assistant Function**

In [ ]:
def smart_assistant():
    # Replace with actual folders
    music_folder = "D:/Music"
    video_folder = "D:/Videos"

    music_player = MusicPlayer(music_folder)
    video_player = VideoPlayer(video_folder)
    youtube = YouTube()

    print("Welcome to Smart Assistant!")

    while True:
        user_input = input("What do you want to do (or type 'exit')? ").strip().lower()

        if user_input in ("exit", "quit", "close"):
            print(" Exiting Smart Assistant.")
            break

        elif match_command(user_input, ["music", "audio", "songs", "audio player", "music player"]):
            music_player.play_music()

        elif match_command(user_input, ["video", "videos", "movie", "film", "video player"]):
            video_player.play_video()

        elif match_command(user_input, ["youtube", "yt", "search youtube"]):
            youtube.search()

        elif match_command(user_input, ["phone", "call", "contacts", "calling"]):
            PhoneCall()

        elif "player" in user_input:
            while True:
                follow_up = input("Do you want music or video player? Or do you want to exit? ").strip().lower()
                if "music" in follow_up or "audio" in follow_up:
                    music_player.play_music()
                    break
                elif "video" in follow_up:
                    video_player.play_video()
                    break
                elif "exit" in follow_up or "quit" in follow_up or "close" in follow_up:
                    print(" Exiting Smart Assistant.")
                    break
                else:
                    print("Didn't understand. Please say music or video.")

                

        else:
            print("I didn't understand that. Try something like 'music', 'video', or 'YouTube'.")